# Schulze's Method of Voting
This notebook implements Schulze's method of voting to combine a set of ranked lists into an overall ranked list. More information on this method and topic can be found at the following links:

 - https://arxiv.org/abs/1804.02973
 - https://en.wikipedia.org/wiki/Schulze_method
 - https://en.wikipedia.org/wiki/Comparison_of_electoral_systems

In [1]:
import numpy as np
import pandas as pd
from ipydatagrid import DataGrid

## Test Data
Test data from a [Google sheet](https://docs.google.com/spreadsheets/d/1NUoOme-YQHA8cfnbPjtHBt13bkZPQkQei3cPAQ611I8/edit?usp=sharing) which has all of its tabs published as CSV files. Some of these sets have the names of voters or other information in the first row, and others have the number of matching ballots in the first row, hence the two different methods for importing votes. test1 and test2 are from from the ranked-pairs and Schulze method examples on Wikipedia, respectively. test3 and test4 are my own tests that are ostensibly amenable to "common-sense" inspection. The "example" sets are examples used in the original Schulze manuscript.

In [2]:
test1 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=1591363425&single=true&output=csv'
test2 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=541601874&single=true&output=csv'
test3 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=2108916501&single=true&output=csv'
test4 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=662509174&single=true&output=csv'
example1 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=721130038&single=true&output=csv'
example2 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=1181234950&single=true&output=csv'
example3 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=1865810740&single=true&output=csv'
example4 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=920600198&single=true&output=csv'
example6 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=753260081&single=true&output=csv'
example7 = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRGPvxWcqyo86JDfGhCn9laHlLHUrMEb8jAqdBFNN_uimR95VxE9JCPpBxr4WWaeLeAPuLHCzGDbQ8y/pub?gid=959087225&single=true&output=csv'

In [3]:
def import_votes(url, first_row='voters'):
    if first_row=='voters':
        votes = pd.read_csv(url)
        #votes.columns = [(f'B{n:02d}') for n in range(votes.shape[1])]
    elif first_row=='count':
        vote_counts = pd.read_csv(url, header=None)
        votes = np.tile(vote_counts.iloc[1:,0].values, [np.int64(vote_counts.iloc[0,0]),1])
        if vote_counts.shape[1]>1:
            for i in np.arange(1,vote_counts.shape[1]):
                votes = np.append(votes, np.tile(vote_counts.iloc[1:,i].values, [np.int64(vote_counts.iloc[0,i]),1]), axis=0)
        votes = pd.DataFrame(votes.T)
    else:
        raise ValueError('Pass voters or count as first_row.')
    return votes

In [4]:
votes = import_votes(test4, first_row='voters')
votes

,Tim,John,Cindy,Fabian
0,Jenny,Jenny,Jenny,Bill
1,Betty,Betty,Betty,Kim
2,Kim,Kim,Kim,Darlene
3,Darlene,Darlene,Sandy,Sandy


In [5]:
candidates = np.unique(votes.values.reshape(votes.shape[0]*votes.shape[1]))
candidates

array(['Betty', 'Bill', 'Darlene', 'Jenny', 'Kim', 'Sandy'], dtype=object)

In [6]:
len(candidates)

6

## Interactive Data
Edit the names of voters and the number of candidates to create an empty editable table of votes, then edit the cells and continue from there.

In [7]:
voters = ['Tim', 'John', 'Cindy', 'Fabian']
number_of_candidates = 7

In [8]:
votes = pd.DataFrame(columns=voters, index=np.arange(1,number_of_candidates+1))
votes.replace(np.nan, '', inplace=True)

In [9]:
datagrid = DataGrid(votes, editable=True, index_name='rank', layout={'height': '200px'})
datagrid

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [10]:
votes = datagrid.data
votes

,Tim,John,Cindy,Fabian
rank,,,,
1,,,,
2,,,,
3,,,,
4,,,,
5,,,,
6,,,,
7,,,,


In [11]:
candidates = np.unique(votes.values.reshape(votes.shape[0]*votes.shape[1]))
candidates

array([''], dtype=object)

## Preference Matrix
The matrix of pair-wise preferences indicates the number of ballots that have candidate A over candidate B for each set of candidates.

In [12]:
def get_pref_count(votes, candidate_a, candidate_b):
    count = 0
    for i in range(votes.shape[1]):
        try:
            apos = votes.iloc[:,i][votes.iloc[:,i]==candidate_a].index.values[0]
        except:
            apos = None
        try:
            bpos = votes.iloc[:,i][votes.iloc[:,i]==candidate_b].index.values[0]
        except:
            bpos = None
            
        if apos == None:
            continue
        if bpos == None:
            if apos != None:
                count += 1
                continue
        if apos < bpos:
            count +=1
    return count

In [13]:
def get_pref_matrix(votes):
    candidates = np.unique(votes.values.reshape(votes.shape[0]*votes.shape[1]))
    pref_vals = np.zeros([len(candidates), len(candidates)], dtype=np.int64)
    i = 0
    j = 0
    for candidate_a in candidates:
        for candidate_b in candidates:
            pref_vals[i,j] = get_pref_count(votes, candidate_a, candidate_b)
            j += 1
        i += 1
        j = 0
    return pd.DataFrame(pref_vals, columns=candidates, index=pd.Index(candidates))

In [14]:
pref_matrix = get_pref_matrix(votes)
pref_matrix

,
,0


## Path Strengths
The "path strengths" are the weakest link of the strongest path between each candidate, in the directed graph of pairwise preferences that can be generated from the preference matrix. For a relatively simple explanation of this, see https://en.wikipedia.org/wiki/Schulze_method.

In [15]:
def get_path_strengths(pref_matrix):
    pref_vals = pref_matrix.values
    candidates = pref_matrix.columns.values
    
    p = pref_vals * 0
    for i in range(len(candidates)):
        for j in range(len(candidates)):
            if pref_vals[i,j] > pref_vals[j,i]:
                p[i,j] = pref_vals[i,j]

    for i in range(len(candidates)):
        for j in range(len(candidates)):
            if i != j:
                for k in range(len(candidates)):
                    if i != j and j != k:
                        p[j,k] = np.max([p[j,k], min([p[j,i], p[i,k]])])
                        
    return pd.DataFrame(p, columns=candidates, index=pd.Index(candidates))

In [16]:
path_strengths = get_path_strengths(pref_matrix)
path_strengths

,
,0


## Schulze Winner Matrix
The winner matrix indicates which candidates are preferred over the others, based on the matriix of path strengths.

In [17]:
def get_winner_matrix(path_strengths):
    p = path_strengths.values
    winner = p * 0
    for i in range(len(candidates)):
        for j in range(len(candidates)):
            if i != j:
                if p[i,j] > p[j,i]:
                    winner[i,j] = 1
    return pd.DataFrame(winner, columns=candidates, index=pd.Index(candidates))

In [18]:
winner_matrix = get_winner_matrix(path_strengths)
winner_matrix

,
,0


## Overall Schulze Ranking
Sorting the sum of the winner matrix in the horizontal-direction provides the overall ranking of candidates. In this example, using the Test4 data, Bill and Sandy are tied for last place, which is an interesting outcome. There are no other ambiguities in the results in this example.

In [19]:
final_ranking = pd.Series(np.sum(winner_matrix.values, axis=1), index=candidates).sort_values(ascending=False)
final_ranking

    0
dtype: int64